Наши данные

In [1]:
import pandas as pd
import re

pth = "/home/sondors/Documents/price/BERT_NER/csv/list1.xlsx"

clms = ['PRICE_FRNO', 
    'PRICE_FNAM', 
    'PRICE_CAID', 
    'PRICE_CNAM',  
    'PRICE_TMNO', 
    'PRICE_PRCN', 
    'PRICE_VERT', 
    'PRICE_PCOD', 
    'date', 
    'PRICE_IMGN', 
    'PRICE_LOCD', 
    'lineNumber',
    'PRICE_PRDS',
    'PRICE_DPRN',
    'PRICE_PTYP',
    'PRICE_PROMO',
    'PRICE_CURR',
    'PRICE_LOCA',
    'PRICE_WARR',
    'PRICE_RSLT',
    'PRICE_PLID',
    'PRICE_SALES',
    'PRICE_DTSH',
    'PRICE_BARCODE',
    'комментарий',
    'kpl',
    'PRICE_PARAM',
    'PRICE_NOTE']

clms_hz = ['kpl',
    'PRICE_NOTE',
    'PRICE_PARAM',
    'XL',
    'RFT']

df = pd.read_excel(pth)
df = df.drop(columns=clms + clms_hz)
df = df.dropna(subset=['PRICE_NAME', 'Ширина', 'Высота', 'Радиус', 'Инд. Скорост', 'Линейка', 'ppl'])
df = df.reset_index(drop=True)
df

,PRICE_NAME,Ширина,Высота,Радиус,Инд. Скорост,Линейка,ppl
0,12067 Bridgestone Автошина R20 275/50 Bridgest...,275,50,20,113R,Blizzak DM-V2,Bridgestone
1,12088 BRIDGESTONE Автошина R20 255/50 Bridgest...,255,50,20,109T,Blizzak DM-V2,Bridgestone
2,13/175/70 Bridgestone Ice Cruiser 7000S 82T ш,175,70,17,82S,Ice Cruiser 7000S,Bridgestone
3,13602 BRIDGESTONE Автошина R18 275/35 Bridgest...,275,35,18,95S,Blizzak Ice,Bridgestone
4,13608 Bridgestone Автошина R18 255/45 Bridgest...,255,45,18,99S,Blizzak Ice,Bridgestone
...,...,...,...,...,...,...,...
20151,R22 275/40 107Y XL Bridgestone Alenza 001 * BR...,275,40,22,107Y,Alenza,BRIDGESTONE
20152,"Бриджстоун 2155017 S 91 REVO-GZ, BRIDGESTONE 1...",215,50,17,91S,Blizzak Revo-GZ,Bridgestone
20153,"Бриджстоун 2155517 v 94 t001, BRIDGESTONE 1186...",215,55,17,94V,Turanza T001,Bridgestone
20154,шинаBridgestone LT245/75R16 120Q Dueler M/T 674,245,75,16,120Q,Dueler,Bridgestone


In [2]:
print(df.keys())

Index(['PRICE_NAME', 'Ширина', 'Высота', 'Радиус', 'Инд. Скорост', 'Линейка',
       'ppl'],
      dtype='object')


In [3]:
def process_text(input_text):
    def separate_letters_and_numbers(input_text):
        # Используем регулярное выражение для поиска сочетаний букв и цифр
        pattern = re.compile(r'(\D+|\d+)')
        
        # Используем findall для нахождения всех сочетаний
        matches = pattern.findall(input_text)
        # Возвращаем строку с пробелами между буквами и цифрами
        return ' '.join(matches)

    processed_text = separate_letters_and_numbers(input_text)    
    processed_text = processed_text.replace("|", " | ")
    processed_text = processed_text.replace("(", " ( ")
    processed_text = processed_text.replace(")", " ) ")
    processed_text = processed_text.replace("[", " [ ")
    processed_text = processed_text.replace("]", " ] ")
    # Убираем повторяющиеся пробелы
    processed_text = re.sub(r'  +', ' ', processed_text)
    return processed_text

def create_bio_tags(row):
    def remove_word(line, word):
        # Преобразуем обе строки в нижний регистр для сравнения без учета регистра
        lower_line = line.lower()
        lower_word = word.lower()

        # Проверяем, содержится ли подстрока в строке
        if lower_word in lower_line:
            # Используем регулярное выражение для удаления подстроки (с учетом регистра)
            line = re.sub(fr'\b{re.escape(word)}\b', '', line, flags=re.IGNORECASE).strip()

        return line
        
    name = str(row['offer'])
    width = str(row['width'])
    height = str(row['height'])
    radius = str(row['radius'])
    brand = row['brand']
    line = str(row['line'])
    v_ind = str(row['v_ind'])

    # Создаем пустой список для хранения тегов
    tags = ['O'] * len(name.split())
    Begin_line = True
    Begin_v_ind = True
    Begin_radius = True
    # Проверяем каждое слово в наименовании и устанавливаем соответствующий тег
    for i, word in enumerate(name.split()):
        if word == '/':
            continue
        
        #   width
        if word == width:
            tags[i] = 'B-width'
            width = width.replace(word,"")

        #   height
        elif word == height:
            tags[i] = 'B-height'
            height = height.replace(word,"")

        #   radius    
        elif word.upper() in radius.upper():
            if radius.replace(" ","") == "":
                continue

            if Begin_radius:
                tags[i] = 'B-radius'
                Begin_radius = False
            else:
                tags[i] = 'I-radius'
            radius = radius.replace(word.lower(),"").replace(word.upper(),"")

        #   brand
        elif word in brand:
            tags[i] = 'B-brand'

        #   v_ind
        elif word.upper() in v_ind.upper():
            if v_ind.replace(" ","") == "":
                continue

            if Begin_v_ind:
                tags[i] = 'B-v_ind'
                Begin_v_ind = False
            else:
                tags[i] = 'I-v_ind'  
            v_ind = v_ind.replace(word.lower(),"").replace(word.upper(),"")  

        #   line
        elif word.upper() in line.upper() or re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")) in line.upper():
            if len(re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")))<3 and Begin_line:
                continue
            if line.replace(" ","") == "":
                continue
            
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'
            line = min(remove_word(line, word), line.replace(word,""))

    # Собираем теги в строку
    return ','.join(tags)

def change_columns(df):   
    new_df = pd.DataFrame()

    df = df.rename(columns={'PRICE_NAME': 'offer'})
    df = df[df['offer'].notna()]

    new_df['offer'] = df['offer'].astype(str)
    new_df['width'] = df['Ширина'].astype(str)
    new_df['height'] = df['Высота'].astype(str)
    new_df['radius'] = df['Радиус'].astype(str)
    new_df['v_ind'] = df['Инд. Скорост'].astype(str)
    new_df['brand'] = df['ppl'].astype(str)
    new_df['line'] = df['Линейка'].astype(str)
    
    def process_column(row, column_name):
        # Проверка, является ли значение в "offer" подстрокой значения из указанной колонки
        contains_column_value = str(row['offer']).find(str(row[column_name])) != -1
        
        # Если значение из колонки не найдено в "offer"
        if not contains_column_value:
            # Замена "." на "," в соответствующей строке из columns_to_process
            row[column_name] = str(row[column_name]).replace('.', ',')
            
            # Повторная проверка
            contains_column_value = str(row['offer']).find(str(row[column_name])) != -1

            # Если проверка не пройдена, удаляем строку из таблицы
            if not contains_column_value:
                return None
        return row
    
    def process_radius1(txt):
        return txt.replace("r", "").replace("R", "")
    
    new_df['radius'] = new_df['radius'].apply(process_radius1)
    columns_to_process = ['width', 'height', 'radius']
    for column in columns_to_process:
        new_df = new_df.apply(process_column, args=(column,), axis=1)

    # Удаляем строки, в которых проверка не пройдена
    new_df = new_df.dropna().reset_index(drop=True)

    def process_radius(txt):
        return "R"+txt
    
    new_df['radius'] = new_df['radius'].apply(process_radius)
    new_df['offer'] = new_df['offer'].apply(process_text)
    new_df['brand'] = new_df['offer'].apply(extract_Bridgestone)
    new_df['BIO_Tags'] = new_df.apply(create_bio_tags, axis=1)

    keywords = ['width', 'height', 'radius', 'v_ind', 'brand', 'line']

    # Проверяем вхождение каждого ключевого слова в каждую строку
    for keyword in keywords:
        new_df = new_df[new_df['BIO_Tags'].str.contains(keyword, case=False)]

    # Переиндексируем DataFrame после удаления строк
    new_df = new_df.reset_index(drop=True)

    new_df = new_df[new_df['v_ind'].apply(lambda x: len(str(x)) <= 5)]
    # Переиндексация DataFrame после удаления строк
    new_df = new_df.reset_index(drop=True)

    return new_df

def extract_Bridgestone(text):
    brands = re.findall(r'Bridgestone|BRIDGESTONE|Бриджстоун|bridgestone|Бриджстон|Bridgeston|бриджстоун|бриджстон', text)
    return brands

new_df = change_columns(df)
new_df

,offer,width,height,radius,v_ind,brand,line,BIO_Tags
0,12067 Bridgestone Автошина R 20 275 / 50 Bridg...,275,50,R20,113R,"[Bridgestone, Bridgestone]",Blizzak DM-V2,"O,B-brand,O,B-radius,I-radius,B-width,O,B-heig..."
1,12088 BRIDGESTONE Автошина R 20 255 / 50 Bridg...,255,50,R20,109T,"[BRIDGESTONE, Bridgestone]",Blizzak DM-V2,"O,B-brand,O,B-radius,I-radius,B-width,O,B-heig..."
2,13602 BRIDGESTONE Автошина R 18 275 / 35 Bridg...,275,35,R18,95S,"[BRIDGESTONE, Bridgestone]",Blizzak Ice,"O,B-brand,O,B-radius,I-radius,B-width,O,B-heig..."
3,13608 Bridgestone Автошина R 18 255 / 45 Bridg...,255,45,R18,99S,"[Bridgestone, Bridgestone]",Blizzak Ice,"O,B-brand,O,B-radius,I-radius,B-width,O,B-heig..."
4,13612 BRIDGESTONE Автошина R 19 255 / 40 Bridg...,255,40,R19,96S,"[BRIDGESTONE, Bridgestone]",Blizzak Ice,"O,B-brand,O,B-radius,I-radius,B-width,O,B-heig..."
...,...,...,...,...,...,...,...,...
19084,R 21 295 / 40 111 V XL Bridgestone Blizzak LM ...,295,40,R21,111V,"[Bridgestone, BRIDGESTONE]",Blizzak LM005,"B-radius,I-radius,B-width,O,B-height,B-v_ind,I..."
19085,R 21 315 / 35 111 V XL Bridgestone Blizzak LM ...,315,35,R21,111V,"[Bridgestone, BRIDGESTONE]",Blizzak LM005,"B-radius,I-radius,B-width,O,B-height,B-v_ind,I..."
19086,R 22 275 / 40 107 V XL Bridgestone Blizzak LM ...,275,40,R22,107V,"[Bridgestone, BRIDGESTONE]",Blizzak LM005,"B-radius,I-radius,B-width,O,B-height,B-v_ind,I..."
19087,R 22 275 / 40 107 Y XL Bridgestone Alenza 001 ...,275,40,R22,107Y,"[Bridgestone, BRIDGESTONE]",Alenza,"B-radius,I-radius,B-width,O,B-height,B-v_ind,I..."


In [4]:



example1 = {'offer': 'r 20 28550 michelin x-ice snow suv 116 t xl зима, MICHELIN 561981',
 'width': '285',
 'height': '50',
 'radius': 'R20',
 'v_ind': '116T',
 'brand': ['michelin', 'MICHELIN'],
 'line': 'X-Ice Snow SUV'}

example1_BIO = create_bio_tags(example1)

print(example1['offer'],'\n')
offer = example1['offer'].split()
BIO_Tags = example1_BIO.split(',')

for i in range(len(offer)):
    print(f"{offer[i]}: {BIO_Tags[i]}")

r 20 28550 michelin x-ice snow suv 116 t xl зима, MICHELIN 561981 

r: B-radius
20: I-radius
28550: O
michelin: B-brand
x-ice: B-line
snow: I-line
suv: I-line
116: B-v_ind
t: I-v_ind
xl: O
зима,: O
MICHELIN: B-brand
561981: O


In [6]:
index = 18317
example = dict(new_df.iloc[index])
print(example['offer'],'\n')
offer = example['offer'].split()
BIO_Tags = example['BIO_Tags'].split(',')

for i in range(len(offer)):
    print(f"{offer[i]}: {BIO_Tags[i]}")
example

Bridgestone Шина Blizzak LM 005 215 / 55 R 18 99 V Без шипов 

Bridgestone: B-brand
Шина: O
Blizzak: B-line
LM: I-line
005: I-line
215: B-width
/: O
55: B-height
R: B-radius
18: I-radius
99: B-v_ind
V: I-v_ind
Без: O
шипов: O


{'offer': 'Bridgestone Шина Blizzak LM 005 215 / 55 R 18 99 V Без шипов',
 'width': '215',
 'height': '55',
 'radius': 'R18',
 'v_ind': '99V',
 'brand': ['Bridgestone'],
 'line': 'Blizzak LM005',
 'BIO_Tags': 'B-brand,O,B-line,I-line,I-line,B-width,O,B-height,B-radius,I-radius,B-v_ind,I-v_ind,O,O'}

In [7]:
new_df.to_csv('/home/sondors/Documents/price/BERT_NER/csv/Kazakov_241123/разметка/Bridgestone.csv', sep=';',index=False)

In [ ]:
label2id = {'B-width': 1,
            'B-height': 2, 
            'B-radius': 3, 
            'I-radius': 4,
            'B-brand': 5, 
            'B-line': 6, 
            'I-line': 7,
            'B-v_ind': 8,
            'I-v_ind': 9,
            'O': 0}
    
id2label = {1: 'B-width',
            2: 'B-height', 
            3: 'B-radius', 
            4: 'I-radius',
            5: 'B-brand', 
            6: 'B-line', 
            7: 'I-line',
            8: 'B-v_ind',
            9: 'I-v_ind',
            0: 'O'}